<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Планетарное движение

### Example - Astrophysics
<section class="post-meta">
By Henning G. Hugdal, Magnus H-S Dahle, Håkon W. Ånes and Peter Berg
</section>
Last edited: March 16th 2018 

---

В модулях по [обыкновенным дифференциальным уравнениям](https://www.numfys.net/modules/) мы рассмотрели различные методы решения обыкновенных дифференциальных уравнений (ОДУ) и кратко обсудили точность этих методов. В этом примере мы применим явные и неявные методы Эйлера, а также метод Рунге-Кутты четвертого порядка для вычисления траектории Земли вокруг Солнца. Это покажет, как точность методов влияет на рассчитанную траекторию.

### Уравнения движения

Мы упростим задачу, пренебрегая гравитационной силой на Земле со всех других планет, следовательно, рассматривая только силу между Землей и Солнцем. Тогда второй закон Ньютона дает следующие уравнения движения

$$m\vec{a} = m\ddot{\vec{r}}= -\frac{GMm}{r^3}\vec{r},$$

где $m$ - масса Земли, $M$ - масса Солнца, $r=|\vec{r}|$ - расстояние между Солнцем и Землей, а $G$ - гравитационная постоянная. Мы попытаемся записать это в безразмерной форме, используя разделение Афелия $R$ между Землей и Солнцем в качестве единицы измерения, а период $T$ в качестве единицы времени. Следовательно, мы определяем

$$\tau \equiv\frac{t}{T},\,\rho \equiv \frac{r}{R},\,X \equiv\frac{x}{R}\,\mathrm{and}\,Y\equiv\frac{y}{R}.$$

Вставив это в уравнение движения, разделив общий коэффициент $m$, мы получим

$$\frac{R}{T^2}\frac{d^2\vec{\rho}}{d\tau^2} = -\frac{GM}{R^2\rho^3}\vec{\rho}.$$

Давайте посмотрим, правильно ли мы это сделали, проверив, что оставшаяся константа безразмерна,

$$\left[\frac{GMT^2}{R^3}\right] = \frac{\mathrm{N(m/kg)^2\cdot kg \cdot s^2}}{m^3} = 1.$$
Определим константу
$$C \equiv \frac{GMT^2}{R^3},$$

и получить безразмерные уравнения движения

$$\frac{d^2\vec{\rho}}{d\tau^2} = -C \frac{\vec{\rho}}{\rho^3}.$$

### Численная реализация

Поскольку угловой момент системы в этой задаче сохраняется, мы рассмотрим движение Земли в плоскости, где Солнце неподвижно в начале координат. Следовательно, у нас есть $\vec{\rho} = X\hat{x} + Y\hat{y}$, и мы хотим переписать приведенное выше уравнение в виде четырех ОДУ первого порядка, чтобы использовать упомянутые численные методы. Мы получаем

$$\begin{align}
\frac{dX}{d\tau} &= U,\qquad
\frac{dU}{d\tau} = -C\frac{X}{\rho^3},\\
\frac{dY}{d\tau} &= V,\qquad
\frac{dV}{d\tau} = -C\frac{Y}{\rho^3},
\end{align}$$

где $U$ и $V$-безразмерные скорости в направлении $x$ и $y$, а $\rho=\sqrt{X^2+Y^2}$.

Мы будем использовать начальные условия

$$
\begin{align}
x_0&=R \,&\Rightarrow\, &&&X_0=1,\\
v_0 &= 30.29 \mathrm{ km/s} \,&\Rightarrow\, &&&V_0 = \frac{u T}{R} = 29.29\cdot 10^3 \mathrm{m/s} \frac{365.25 \cdot 24 \cdot 3600 \mathrm{s}}{152.10\cdot 10^9 \mathrm{m}} \approx 6.33,
\end{align}
$$

и $Y_0 = 0$, $U_0 = 0$.

Теперь мы готовы приступить к внедрению различных методов. Ниже мы будем использовать $r$ вместо $\rho$. Сначала мы импортируем необходимые библиотеки и определяем глобальные константы.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc

#Set common figure parameters:
newparams = {
    'figure.figsize': (16, 5), 'axes.grid': True,
    'lines.linewidth': 1.5, 'font.size': 19, 'lines.markersize' : 10,
    'mathtext.fontset': 'stix', 'font.family': 'STIXGeneral'}
plt.rcParams.update(newparams)

T = 365.25*24*3600  # Orbital period [s]
R = 152.10e9        # Aphelion distance [m]
G = 6.673e-11       # Gravitational constant [N (m/kg)^2]
M = 1.9891e30       # Solar mass [kg]
m = 5.9726e24       # Earth mass [kg]

C = (G*M*T**2)/(R**3)
#print(C)

# Initial conditions
X0 = 1
U0 = 0
Y0 = 0
V0 = 29.29e3*T/R

t_max = 1    # t_max=1 Corresponds to one single complete orbit
dt = 0.0001
N = int(t_max/dt)  # Number of time steps

Давайте теперь воспользуемся различными численными методами для расчета траектории. Чтобы увидеть, насколько хороши эти методы, будем ориентироваться на физические принципы:
- Энергия должна быть сохранена, и, следовательно, орбита должна быть замкнута.
- Мы знаем точку перигелия (минимальное расстояние до Солнца), $\rho_\mathrm{перигелий} = 0,967$, и можем проверить, насколько близка траектория к этой точке.

Давайте начнем!

### Явный метод Эйлера

Сначала мы создаем функцию, которая возвращает RHS четырех приведенных выше ОДУ, прежде чем реализовать явный метод Эйлера.

In [ ]:
def RHS(x, y, u, v):
    """ Returns the time derivatives of X, Y, U and V. Also returns the total energy E of the system."""
    
    r = np.sqrt(x**2 + y**2)
    dxdt = u
    dydt = v
    dudt = -C*x/r**3
    dvdt = -C*y/r**3
    
    E = 0.5*(u**2+v**2)-C/r
    
    return (dxdt, dydt, dudt, dvdt, E)

X = np.zeros(N)
Y = np.zeros(N)
U = np.zeros(N)
V = np.zeros(N)
E = np.zeros(N-1)  # Total energy/mass

X[0] = X0
V[0] = V0

for n in range(N-1):
    (dX, dY, dU, dV, E[n]) = RHS(X[n], Y[n], U[n], V[n])
    X[n+1] = X[n] + dt*dX
    Y[n+1] = Y[n] + dt*dY
    U[n+1] = U[n] + dt*dU
    V[n+1] = V[n] + dt*dV
    
    
# If t_max was set to exactly one period, it will be interesting
# to investigate whether or not the orbit is closed (planet returns
# to its starting position)
if(t_max==1):  
    # Find offset
    print("\nSmall offset indicates closed orbit")
    print("Offset in 'x': %0.3e - %0.7e = %0.7e" % (X[0], X[-1], X[-1]-X[0]))
    print("Offset in 'y': %0.3e - %0.7e = %0.7e" % (Y[0], Y[-1], Y[-1]-Y[0]))
    print("Total offset: %0.3e" % np.sqrt((X[0]-X[-1])**2+(Y[0]-Y[-1])**2))
    
    # Find perihelion seperation:
    r_perihelion = abs(min(Y))
    print("\nThe perihelion seperation is %0.3f, compared to 0.967." % r_perihelion)

    
plt.figure()
plt.plot(X, Y, 'g', [0], [0], 'ro')
plt.title("Explicit Euler")
plt.xlabel(r"$x$")
plt.ylabel(r"$y$")
plt.grid()

plt.figure()
plt.plot(E)
plt.title('Energy per unit mass')
plt.ylabel("Energy")
plt.xlabel(r"$n$")
plt.grid();

### Неявный метод Эйлера

При использовании неявного метода Эйлера мы оцениваем RHS ОДУ на каждом последующем временном шаге. Отсюда мы получаем следующие уравнения
$$\begin{align}
X_{n+1} &= X_{n} + \Delta t U_{n+1},\\
U_{n+1} &= U_n - \Delta t C\frac{X_{n+1}}{\rho_{n+1}^3},\\
Y_{n+1} &= Y_{n} + \Delta t V_{n+1},\\
V_{n+1} &= V_n - \Delta t C\frac{Y_{n+1}}{\rho_{n+1}^3}.
\end{align}$$
Мы будем использовать решатель,

    scipy.optimize.root,

чтобы найти значения на новом временном шаге в приведенных выше уравнениях.

In [ ]:
from scipy.optimize import root

def func(X, X0):
    """ Function returning the difference between the RHS and LHS of the four first order ODEs above. 
    
    X:     vector of new coordinates
    X0:    vector of old coordinates
    
    """
    
    X_ = X[0]
    Y_ = X[1]
    U_ = X[2]
    V_ = X[3]
    
    X0_ = X0[0]
    Y0_ = X0[1]
    U0_ = X0[2]
    V0_ = X0[3]
    
    r = np.sqrt(X_**2 + Y_**2)
    
    return [X_ - (X0_ + dt*U_),
            Y_ - (Y0_ + dt*V_),
            U_ - (U0_ - dt*C*X_/r**3),
            V_ - (V0_ - dt*C*Y_/r**3)]
    

X = np.zeros(N)
Y = np.zeros(N)
U = np.zeros(N)
V = np.zeros(N)

X[0] = X0
V[0] = V0

for n in range(N-1):
    X0_ = np.array((X[n], Y[n], U[n], V[n]))
    sol = root(func, x0=X0_, args=X0_, jac=False)
    
    X[n+1] = sol.x[0]
    Y[n+1] = sol.x[1]
    U[n+1] = sol.x[2]
    V[n+1] = sol.x[3]


# If t_max was set to exactly one period, it will be interesting
# to investigate whether or not the orbit is closed (planet returns
# to its starting position)
if(t_max==1):  
    # Find offset
    print("\nSmall offset indicates closed orbit")
    print("Offset in 'x': %0.3e - %0.7e = %0.7e" % (X[0], X[-1], X[-1]-X[0]))
    print("Offset in 'y': %0.3e - %0.7e = %0.7e" % (Y[0], Y[-1], Y[-1]-Y[0]))
    print("Total offset: %0.3e" % np.sqrt((X[0]-X[-1])**2+(Y[0]-Y[-1])**2))

    # Find perihelion seperation:
    r_perihelion = abs(min(Y))
    print("\nThe perihelion seperation is %0.3f, compared to 0.967." % r_perihelion)
    
plt.figure()
plt.plot(X, Y, 'g', [0], [0], 'ro')
plt.title("Implicit Euler")
plt.xlabel(r"$x$")
plt.ylabel(r"$y$")
plt.grid()

# Calculate energy
r = np.sqrt(X**2 + Y**2)
E = 0.5*(U**2+V**2)-C/r

plt.figure()
plt.plot(E)
plt.title('Energy per unit mass')
plt.ylabel("Energy")
plt.xlabel(r"$n$")
plt.grid();

### Метод Рунге-Кутты Четвертого Порядка

Далее мы решим эту систему в последний раз, используя метод Рунге-Кутты четвертого порядка (RKM4). Поскольку RKM4 использует тестовые точки, мы разделим четыре уравнения на отдельные функции.

In [ ]:
def F(X_, Y_, U_, V_): # dX/dtau
    return U_

def G(X_, Y_, U_, V_): # dY/dtau
    return V_

def H(X_, Y_, U_, V_): # dU/dtau
    return -C * (  X_/( (np.sqrt(X_**2 + Y_**2) )**3)  )

def I(X_, Y_, U_, V_): # dV/dtau
    return -C * (  Y_/( (np.sqrt(X_**2 + Y_**2) )**3)  )

X_4RK = np.zeros(N)
Y_4RK = np.zeros(N)
U_4RK = np.zeros(N)
V_4RK = np.zeros(N)
E_4RK = np.zeros(N-1)  # Total energy/mass

X_4RK[0] = X0
V_4RK[0] = V0

for n in range(N-1):
    
    k_x1 = dt * F( X_4RK[n], Y_4RK[n], U_4RK[n], V_4RK[n] )
    k_y1 = dt * G( X_4RK[n], Y_4RK[n], U_4RK[n], V_4RK[n] )
    k_u1 = dt * H( X_4RK[n], Y_4RK[n], U_4RK[n], V_4RK[n] )
    k_v1 = dt * I( X_4RK[n], Y_4RK[n], U_4RK[n], V_4RK[n] )
    
    k_x2 = dt * F( X_4RK[n] + k_x1/2, Y_4RK[n] + k_y1/2, U_4RK[n] + k_u1/2, V_4RK[n] + k_v1/2 )
    k_y2 = dt * G( X_4RK[n] + k_x1/2, Y_4RK[n] + k_y1/2, U_4RK[n] + k_u1/2, V_4RK[n] + k_v1/2 )
    k_u2 = dt * H( X_4RK[n] + k_x1/2, Y_4RK[n] + k_y1/2, U_4RK[n] + k_u1/2, V_4RK[n] + k_v1/2 )
    k_v2 = dt * I( X_4RK[n] + k_x1/2, Y_4RK[n] + k_y1/2, U_4RK[n] + k_u1/2, V_4RK[n] + k_v1/2 )
    
    k_x3 = dt * F( X_4RK[n] + k_x2/2, Y_4RK[n] + k_y2/2, U_4RK[n] + k_u2/2, V_4RK[n] + k_v2/2 )
    k_y3 = dt * G( X_4RK[n] + k_x2/2, Y_4RK[n] + k_y2/2, U_4RK[n] + k_u2/2, V_4RK[n] + k_v2/2 )
    k_u3 = dt * H( X_4RK[n] + k_x2/2, Y_4RK[n] + k_y2/2, U_4RK[n] + k_u2/2, V_4RK[n] + k_v2/2 )
    k_v3 = dt * I( X_4RK[n] + k_x2/2, Y_4RK[n] + k_y2/2, U_4RK[n] + k_u2/2, V_4RK[n] + k_v2/2 )
    
    k_x4 = dt * F( X_4RK[n] + k_x3, Y_4RK[n] + k_y3, U_4RK[n] + k_u3, V_4RK[n] + k_v3 )
    k_y4 = dt * G( X_4RK[n] + k_x3, Y_4RK[n] + k_y3, U_4RK[n] + k_u3, V_4RK[n] + k_v3 )
    k_u4 = dt * H( X_4RK[n] + k_x3, Y_4RK[n] + k_y3, U_4RK[n] + k_u3, V_4RK[n] + k_v3 )
    k_v4 = dt * I( X_4RK[n] + k_x3, Y_4RK[n] + k_y3, U_4RK[n] + k_u3, V_4RK[n] + k_v3 )
    
    X_4RK[n+1] = X_4RK[n] + k_x1/6 + k_x2/3 + k_x3/3 + k_x4/6
    Y_4RK[n+1] = Y_4RK[n] + k_y1/6 + k_y2/3 + k_y3/3 + k_y4/6
    U_4RK[n+1] = U_4RK[n] + k_u1/6 + k_u2/3 + k_u3/3 + k_u4/6
    V_4RK[n+1] = V_4RK[n] + k_v1/6 + k_v2/3 + k_v3/3 + k_v4/6
    
    E_4RK[n] = 0.5*(U_4RK[n+1]**2+V_4RK[n+1]**2)-C/np.sqrt(X_4RK[n+1]**2 + Y_4RK[n+1]**2)

    
# If t_max was set to exactly one period, it will be interesting
# to investigate whether or not the orbit is closed (planet returns
# to its starting position)
if(t_max==1):
    # Find offset
    print("\nSmall offset indicates closed orbit")
    print("Offset in 'x': %0.3e - %0.7e = %0.7e" % (X_4RK[0], X_4RK[-1], X_4RK[N-1]-X_4RK[0]))
    print("Offset in 'y': %0.3e - %0.7e = %0.7e" % (Y_4RK[0], Y_4RK[-1], Y_4RK[N-1]-Y_4RK[0]))
    print("Total offset: %0.3e" % np.sqrt((X_4RK[0]-X_4RK[-1])**2+(Y_4RK[0]-Y_4RK[-1])**2))

    # Find perihelion seperation:
    r_perihelion = abs(min(Y_4RK))
    print("\nThe parahelion seperation is %0.3f, compared to 0.967." % r_perihelion)

plt.figure()
plt.title('4th order Runge-Kutta')
plt.plot(X_4RK, Y_4RK, 'g', [0], [0], 'ro')
plt.xlabel(r"$x$")
plt.ylabel(r"$y$")
plt.grid()

plt.figure()
plt.title('Energy per unit mass')
plt.plot(E_4RK)
plt.ylabel("Energy")
plt.xlabel(r"$n$")
plt.grid()

Из приведенных выше результатов мы видим, что метод Рунге-Кутты на самом деле является наиболее точным: в то время как график энергии/массы для двух первых методов показывает большие отклонения, энергия/масса близка к постоянной для RKM4. Это приводит к почти замкнутой орбите. Это не должно удивлять, поскольку RKM4 реализован как метод 4-го порядка и, следовательно, должен превосходить явные и неявные методы Эйлера, которые являются методами 1-го порядка, с точки зрения точности.

___
Числовые значения из http://nssdc.gsfc.nasa.gov/planetary/factsheet/earthfact.html